# OCR 모델 이용해 text detection & text recognition 작업 수행

In [1]:
# import torch
# torch.cuda.is_available()
# torch.cuda.get_device_name(0)

In [2]:
import pandas as pd
import numpy as np
import os
import json
import pprint
import glob

import tensorflow as tf
import cv2
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

import paddleocr
from paddleocr import PaddleOCR, draw_ocr
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

## 데이터 구조 확인하기

### 데이터 목록이 제대로 있는지 확인하는 코드

In [3]:
# JSON 파일 목록 정리
basic_path = 'data/HW_OCR/4.Validation/'
backgrounds = ['P.Paper/', 'T.Tablet/']
contents = ['R.Free/', 'O.Form/']
json_files_list = []
png_files_list = []

for background in backgrounds:
    for content in contents:
        tmp = []
        path = basic_path + background + content
        tmp = list(map(lambda x: path + str(x), filter(lambda x: x.endswith('.json'), os.listdir(path))))
        json_files_list.append(tmp)
        print('{}.json: {}개'.format(path, len(tmp)))
        
        tmp = list(map(lambda x: path + str(x), filter(lambda x: x.endswith('.png'), os.listdir(path))))
        png_files_list.append(tmp)
        print('{}.png: {}개'.format(path, len(tmp)))
        print()

data/HW_OCR/4.Validation/P.Paper/R.Free/.json: 35535개
data/HW_OCR/4.Validation/P.Paper/R.Free/.png: 35535개

data/HW_OCR/4.Validation/P.Paper/O.Form/.json: 43170개
data/HW_OCR/4.Validation/P.Paper/O.Form/.png: 43170개

data/HW_OCR/4.Validation/T.Tablet/R.Free/.json: 4789개
data/HW_OCR/4.Validation/T.Tablet/R.Free/.png: 4789개

data/HW_OCR/4.Validation/T.Tablet/O.Form/.json: 12525개
data/HW_OCR/4.Validation/T.Tablet/O.Form/.png: 12525개



In [4]:
cnt_P_AIHUB = 87205
cnt_T_AIHUB = 19314
cnt_AIHUB = cnt_P_AIHUB + cnt_T_AIHUB

P1 = len(json_files_list[0]) + len(json_files_list[1])
T1 = len(json_files_list[2]) + len(json_files_list[3])
print('갖고 있는 json 데이터 목록')
print('Paper: {}, Table: {}, total: {}'.format(P1, T1, P1+T1))
print('데이터 사이트와 내가 갖고 있는 데이터 목록 갯수와의 차이')
print('Paper: {}, Table: {}, total: {}'.format(cnt_P_AIHUB - P1, cnt_T_AIHUB - T1, cnt_AIHUB - (P1+T1)))

print()

P2 = len(png_files_list[0]) + len(png_files_list[1])
T2 = len(png_files_list[2]) + len(png_files_list[3])
print('갖고 있는 png 데이터 목록')
print('Paper: {}, Table: {}, total: {}'.format(P2, T2, P2+T2))
print('데이터 사이트와 내가 갖고 있는 데이터 목록 갯수와의 차이')
print('Paper: {}, Table: {}, total: {}'.format(cnt_P_AIHUB - P2, cnt_T_AIHUB - T2, cnt_AIHUB - (P2+T2)))

갖고 있는 json 데이터 목록
Paper: 78705, Table: 17314, total: 96019
데이터 사이트와 내가 갖고 있는 데이터 목록 갯수와의 차이
Paper: 8500, Table: 2000, total: 10500

갖고 있는 png 데이터 목록
Paper: 78705, Table: 17314, total: 96019
데이터 사이트와 내가 갖고 있는 데이터 목록 갯수와의 차이
Paper: 8500, Table: 2000, total: 10500


In [5]:
path = 'data/HW_OCR/4.Validation/P.Paper/R.Free/'
file = 'IMG_OCR_53_4PR_21045.json'
with open(path + file, 'r', encoding='UTF8') as f:
    data = json.load(f)
pprint.pprint(data)

{'Annotation': {'object_recognition': 0, 'text_language': 0},
 'Dataset': {'category': 0,
             'identifier': 'IMG_OCR_53',
             'label_path': 'HW_OCR/4.Validation/P.Paper/R.Free/',
             'name': '대용량 손글씨 데이터셋',
             'src_path': 'HW_OCR/4.Validation/P.Paper/R.Free/',
             'type': 1},
 'Images': {'acquistion_location': '자체',
            'application_field': '기타',
            'background': 0,
            'data_captured': '2021.09.06',
            'height': 3482,
            'identifier': 'IMG_OCR_53_4PR_21045',
            'media_type': 0,
            'pen_color': 'red',
            'pen_type': 0,
            'type': 'png',
            'width': 2469,
            'writer_age': 47,
            'writer_sex': 1,
            'written_content': 1},
 'bbox': [{'data': '021045',
           'id': 1,
           'x': [1857, 1857, 2262, 2262],
           'y': [153, 248, 153, 248]},
          {'data': '경상북도',
           'id': 2,
           'x': [347, 347, 746, 74

### 결과 데이터 구성 요소

- label에서 가져올 정보
    - id
        - Images > identifier
    - image
        - Images > identifer + '.' + type
        - 두 요소를 .으로 이어 붙어야 한다.
    - age
        - Images > writer_age
    - sex
        - Images > writer_sex
    - background
        - Images > background
        - 종이: 0
        - 타블렛: 1
    - text_origin
        - bbox > data
    - bbox_origin
        - bbox > data, id, x, y 순서로 정렬
        - x와 y에 존재하는 숫자는 좌측 하부, 좌측 상부, 우측 하부, 우측 상부 순서로 정렬되어 있음
        - OCR 결과물과 동일한 순서로 작업을 할 계획이라
            - 좌측 하부, 우측 하부,우측 상부, 좌측 상부 순으로 [x, y] 묶음의 나열로 바꿀것

<br>

- OCR 모델에서 가져올 정보
    - text_ocr
        - ocr 모델이 찾은 모든 글씨
    - bbox_ocr
        - ocr 모델이 찾은 모든 글씨의 bbox 값
        - bbox값이 좌측 하부, 우측 하부,우측 상부, 좌측 상부 순으로 [x, y] 묶음의 나열로 있음
        
<br>

- label과 OCR 모델를 활용해 정합한 정보
    - text
        - 정답 데이터인 text_origin과 ocr 모델이 찾은 데이터인 text_ocr을 비교하여 찾은 데이터 중 정답만을 가져온 것
        - 교집합
    - bbox
        - text 컬럼에 대응되는 bbox값(bbox_ocr 컬럼에서 가져온 것)

In [6]:
# 결과 데이터 컬럼 및 df 생성

col = ['id', 'image', 'age', 'sex', 'background', 'text_origin', 'bbox_origin', 'text_ocr', 'bbox_ocr', 'text', 'bbox']

ocr_df = pd.DataFrame(columns = col)
ocr_df

,id,image,age,sex,background,text_origin,bbox_origin,text_ocr,bbox_ocr,text,bbox


In [7]:
# JSON 파일 로드 함수
def load_json(json_path):
    try:
        encoder = 'utf-8'
        with open(json_path, 'r', encoding = encoder) as f:
            data = json.load(f)
    except:
        try:
            encoder = 'cp949'
            with open(json_path, 'r', encoding = encoder) as f:
                data = json.load(f)
            
        except:
            encoder = 'euc-kr'
            with open(json_path, 'r', encoding = encoder) as f:
                data = json.load(f)
    
    return data

# JSON 파일 목록 정리
basic_path = 'data/HW_OCR/4.Validation/'
backgrounds = ['P.Paper/', 'T.Tablet/']
content = 'R.Free/'
json_files_list = []

for background in backgrounds:
    tmp = []
    path = basic_path + background + content
    tmp = list(map(lambda x: path + str(x), filter(lambda x: x.endswith('.json'), os.listdir(path))))
    json_files_list += tmp
    
print(len(json_files_list))

# JSON 파일 목록 정리
basic_path = 'data/HW_OCR/4.Validation/'
backgrounds = ['P.Paper/', 'T.Tablet/']
contents = ['R.Free/', 'O.Form/']
json_files_list = []
for background in backgrounds:
    tmp = []
    for content in contents:
        path = basic_path + background + content
        tmp = list(map(lambda x: path + str(x), filter(lambda x: x.endswith('.json'), os.listdir(path))))
        json_files_list += tmp
    
print(len(json_files_list))

40324


In [8]:
def jsonNocr2df(json_files_list):
    
    # json 데이터 가져오는 부분
    def json2var(json_path):

        # JSON 파일 변수
        data = load_json(json_path)

        # image 파일 경로 변수
        image_path = 'data/' + data['Dataset']['src_path'] + data['Images']['identifier'] + '.' + data['Images']['type']

        # 진행 상황 확인용 프린트문
#         print(image_path)

        # 각 단어의 바운딩 박스와 텍스트 정보
        texts_list = []
        bounding_box_list = []

        # 전처리된 이미지 리스트
        for word in data['bbox']:
            text = word['data']
            texts_list.append(text)

            # 진행 상황 확인용 프린트문
#             print(text)

            # 좌표들 순서 위치            # paddle_ocr bbox 결과 좌표 순서 위치
            # 1    3                    # 3    2
            # 0    2                    # 0    1
            bounding_box = [[word['x'][0], word['y'][0]],
                            [word['x'][2], word['y'][2]],
                            [word['x'][3], word['y'][3]],
                            [word['x'][1], word['y'][1]]
                           ]
            bounding_box_list.append(bounding_box)

            # 진행 상황 확인용 프린트문
#             print(bounding_box)
        df_id = data['Images']['identifier']
        df_image = image_path
        df_age = data['Images']['writer_age']
        df_sex = data['Images']['writer_sex']
        df_background = data['Images']['background']
        df_text_origin = texts_list
        df_bbox_origin = bounding_box_list
        
        return df_id, df_image, df_age, df_sex, df_background, df_text_origin, df_bbox_origin
    
    # ocr 모델 적용부분
    def png2ocr(img_path):
        ocr_model = PaddleOCR(lang = 'korean')
        result = ocr_model.ocr(img_path, cls = False)

        return result

    # ocr 모델 결과를 리스트 변수로 각각 변환
    def ocr_TSB(ocr_result):
        res = ocr_result[0]

        texts_ocr = [res[i][1][0] for i in range(len(res))]
        scores_ocr = [float(res[i][1][1]) for i in range(len(res))]
        boxes_ocr = [res[i][0] for i in range(len(res))]

        return texts_ocr, scores_ocr, boxes_ocr

    # 정답데이터와 ocr 모델 결과데이터를 비교하여 교집합 내용만 고르는 코드
    def ocrINTERcorrect(correct, texts, boxes):

        texts_inter = []
        boxes_inter = []

        for t, b in zip(texts, boxes):

            for c in correct:
                if t == c:
                    texts_inter.append(t)
                    boxes_inter.append(b)
        if len(texts_inter) == len(boxes_inter):
            return texts_inter, boxes_inter

        else:
            print('ERROR CHECK the CODE')

            print(texts_inter)
            print(len(texts_inter), len(boxes_inter))

            
            
    # json 파일마다 필요한 부분 적용
    for i, json_path in enumerate(json_files_list):

        # 진행 상황 확인용 프린트문
        if i % 100 == 0:
            print('<<{}>>'.format(i), end = ' ')

        # df에 들어갈 전체 변수 선언 부분
        # json 파트
        df_id, df_image, df_age, df_sex, df_background, df_text_origin, df_bbox_origin = json2var(json_path)

        # ocr 파트
        df_text_ocr, score_ocr, df_bbox_ocr = ocr_TSB(png2ocr(df_image))
        df_text, df_bbox = ocrINTERcorrect(df_text_origin, df_text_ocr, df_bbox_ocr)

        ocr_df.loc[len(ocr_df)] = [df_id, df_image, df_age, df_sex, df_background,
                                   df_text_origin, df_bbox_origin,
                                   df_text_ocr, df_bbox_ocr, df_text, df_bbox]


In [9]:
jsonNocr2df(json_files_list[:1000])

0 [2024/08/28 23:20:52] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_mod

[2024/08/28 23:21:11] ppocr DEBUG: dt_boxes num : 69, elapsed : 0.13317179679870605
[2024/08/28 23:21:16] ppocr DEBUG: rec_res num  : 69, elapsed : 5.052396059036255
[2024/08/28 23:21:16] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='

[2024/08/28 23:21:35] ppocr DEBUG: dt_boxes num : 71, elapsed : 0.13327527046203613
[2024/08/28 23:21:41] ppocr DEBUG: rec_res num  : 71, elapsed : 5.3692097663879395
[2024/08/28 23:21:41] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir=

[2024/08/28 23:21:59] ppocr DEBUG: dt_boxes num : 74, elapsed : 0.1421220302581787
[2024/08/28 23:22:05] ppocr DEBUG: rec_res num  : 74, elapsed : 5.226894855499268
[2024/08/28 23:22:05] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='.

[2024/08/28 23:22:23] ppocr DEBUG: dt_boxes num : 71, elapsed : 0.14987421035766602
[2024/08/28 23:22:28] ppocr DEBUG: rec_res num  : 71, elapsed : 5.035765171051025
[2024/08/28 23:22:28] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='

...

[2024/08/29 01:57:14] ppocr DEBUG: dt_boxes num : 73, elapsed : 0.12253904342651367
[2024/08/29 01:57:20] ppocr DEBUG: rec_res num  : 73, elapsed : 5.902201414108276
[2024/08/29 01:57:20] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='

[2024/08/29 01:57:40] ppocr DEBUG: dt_boxes num : 70, elapsed : 0.13188910484313965
[2024/08/29 01:57:45] ppocr DEBUG: rec_res num  : 70, elapsed : 5.327683687210083
[2024/08/29 01:57:45] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='

[2024/08/29 01:58:04] ppocr DEBUG: dt_boxes num : 70, elapsed : 0.14137887954711914
[2024/08/29 01:58:10] ppocr DEBUG: rec_res num  : 70, elapsed : 5.090879440307617
[2024/08/29 01:58:10] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='

[2024/08/29 01:58:28] ppocr DEBUG: dt_boxes num : 71, elapsed : 0.13864612579345703
[2024/08/29 01:58:34] ppocr DEBUG: rec_res num  : 71, elapsed : 5.607041597366333
[2024/08/29 01:58:34] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='

[2024/08/29 01:58:53] ppocr DEBUG: dt_boxes num : 71, elapsed : 0.15650320053100586
[2024/08/29 01:58:58] ppocr DEBUG: rec_res num  : 71, elapsed : 5.132783651351929
[2024/08/29 01:58:58] ppocr DEBUG: Namespace(alpha=1.0, alphacolor=(255, 255, 255), benchmark=False, beta=1.0, binarize=False, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='C:\\Users\\UserK/.paddleocr/whl\\cls\\ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='C:\\Users\\UserK/.paddleocr/whl\\det\\ml\\Multilingual_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='

In [10]:
ocr_df.tail()

,id,image,age,sex,background,text_origin,bbox_origin,text_ocr,bbox_ocr,text,bbox
995,IMG_OCR_53_4PR_05458,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,45,0,0,"[005458, 서울특별시, 세종특별자치시, 경상북도, 울산광역시, 경산시, 창원시...","[[[1850, 147], [2104, 147], [2104, 226], [1850...","[005458, No, 손글씨OCR, 자유필사, DataSheet, 0003, 05...","[[[1868.0, 153.0], [2083.0, 162.0], [2080.0, 2...","[005458, 서울특별시, 세종특별자치시, 울산광역시, 경상북도, 의령군, 경산시...","[[[1868.0, 153.0], [2083.0, 162.0], [2080.0, 2..."
996,IMG_OCR_53_4PR_05459,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,45,0,0,"[005459, 전라북도, 광주광역시, 충청북도, 경기도, 여수시, 사하구, 강서구...","[[[1856, 132], [2251, 132], [2251, 224], [1856...","[605459, No, 손글씨OCR, 자유필사, DataSheet, 00030900...","[[[1882.0, 152.0], [2066.0, 166.0], [2061.0, 2...","[전라북도, 충청북도, 광주광역시, 사하구, 서산시, 강서구, 변산면, 군동면, 조...","[[[401.0, 1044.0], [585.0, 1054.0], [581.0, 11..."
997,IMG_OCR_53_4PR_05460,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,45,0,0,"[005460, 제주특별자치도, 충청북도, 경상북도, 세종특별자치시, 의성군, 관악...","[[[1874, 141], [2106, 141], [2106, 229], [1874...","[005+60, No, 손글씨OCR, 자유필사, DataSheet, -, 00030...","[[[1886.0, 154.0], [2087.0, 154.0], [2087.0, 2...","[제주특별자치도, 충청북도, 경상북도, 관악구, 덕충동, 주생면, 줄포면, 용담삼동...","[[[346.0, 1046.0], [685.0, 1046.0], [685.0, 11..."
998,IMG_OCR_53_4PR_05461,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,45,0,0,"[부산광역시, 대전광역시, 울산광역시, 전라북도, 안산시, 계룡시, 안양시, 창원시...","[[[382, 1012], [679, 1012], [679, 1134], [382,...","[03 546, 손글씨OCR, 자유필사, DataSheet, 0003090089, ...","[[[1925.0, 158.0], [2113.0, 149.0], [2116.0, 2...","[부산광역시, 대전광역시, 울산광역시, 전라북도, 안산시, 상록구, 창원시, 안양시...","[[[405.0, 1031.0], [651.0, 1031.0], [651.0, 11..."
999,IMG_OCR_53_4PR_05462,data/HW_OCR/4.Validation/P.Paper/R.Free/IMG_OC...,45,0,0,"[005462, 대구광역시, 세종특별자치시, 전라남도, 강원도, 통영시, 영도구, ...","[[[1863, 132], [2222, 132], [2222, 231], [1863...","[No, 20546로, 손글씨OCR, 자유필사, DataSheet, -, 00030...","[[[1749.0, 164.0], [1827.0, 164.0], [1827.0, 2...","[대구광역시, 전라남도, 강원도, 통영시, 영도구, 강릉시, 해남군, 임고면, 삼패...","[[[387.0, 1045.0], [644.0, 1045.0], [644.0, 11..."


In [11]:
ocr_df.to_csv('result_df_sample.csv', index = False)

# 전체 데이터 수행하는 부분

In [ ]:
# 결과 데이터 컬럼 및 df 생성

col = ['id', 'image', 'age', 'sex', 'background', 'text_origin', 'bbox_origin', 'text_ocr', 'bbox_ocr', 'text', 'bbox']

ocr_df = pd.DataFrame(columns = col)
ocr_df

In [ ]:
jsonNocr2df(json_files_list[:10000])

In [ ]:
ocr_df.tail()

In [ ]:
jsonNocr2df(json_files_list[10000:20000])

In [ ]:
ocr_df.tail()

In [ ]:
jsonNocr2df(json_files_list[20000:30000])

In [ ]:
ocr_df.tail()

In [ ]:
jsonNocr2df(json_files_list[30000:])

In [ ]:
ocr_df.tail()

In [ ]:
ocr_df.to_csv('result_df.csv', index = False)

# 코드 만들 때 활용하는 부분

## OCR 모델 적용한 결과값 df에 넣기

In [ ]:
# ocr 모델 적용부분
def png2ocr(img_path):
    ocr_model = PaddleOCR(lang = 'korean')
    result = ocr_model.ocr(img_path, cls = False)
    
    return result

# ocr 모델 결과를 리스트 변수로 각각 변환
def ocr_TSB(ocr_result):
    res = ocr_result[0]
    
    texts_ocr = [res[i][1][0] for i in range(len(result[0]))]
    scores_ocr = [float(res[i][1][1]) for i in range(len(result[0]))]
    boxes_ocr = [res[i][0] for i in range(len(result[0]))]
    
    return texts_ocr, scores_ocr, boxes_ocr
    
# 정답데이터와 ocr 모델 결과데이터를 비교하여 교집합 내용만 고르는 코드
def ocrINTERcorrect(correct, texts, boxes):
    
    texts_inter = []
    boxes_inter = []

    for t, b in zip(texts, boxes):

        for c in correct:
            if t == c:
                texts_inter.append(t)
                boxes_inter.append(b)
    if len(texts_inter) == len(boxes_inter):
        return texts_inter, boxes_inter
    else:
        print('ERROR CHECK the CODE')

        print(texts_inter)
        print(len(texts_inter), len(boxes_inter))

# paddle_ocr 모델 적용 후 원하는 값들을 정리하여 각 df에 추가하는 코드
def fillup_df(idx, row):
    
    img_path = row['image']
    
    result = png2ocr(img_path)
    texts_ocr, scores_ocr, boxes_ocr = ocr_TSB(result)
    text, bbox = ocrINTERcorrect(row['text_origin'], texts, boxes)
    
    # 이 부분 추후 잘 적용되는지 확인 필요
    ocr_df.iloc[idx, -4:] = [texts_ocr, boxes_ocr, text, bbox]

# 전체 과정을 적용하는 코드
# 이때, 에러 방지를 위해 df에 적용할 범위를 선택하여 진행함
def png2ocr_df(ocr_df, n = 0, m = 100):
    for idx, row in ocr_df[n:m].iterrows():
    
        # 진행 상황 확인용 프린트문
        if idx % 100 == 0:
            print(idx)
        # 진행 상황 확인용 프린트문
#         print(row['image'])
        fillup_df(idx, row)

In [ ]:
img_path = 'data/' + data['Dataset']['src_path'] + data['Images']['identifier'] + '.' + data['Images']['type']

result = png2ocr(img_path)

In [ ]:
texts, scores, boxes = ocr_TSB(result)

correct = []
for i in data['bbox']:
    correct.append(i['data'])

text, bbox = ocrINTERcorrect(correct, texts, boxes)

print(text)
print(bbox)

In [ ]:
for idx, row in ocr_df[0:3].iterrows():

    img_path = row['image']

    result = png2ocr(img_path)
    texts_ocr, scores_ocr, boxes_ocr = ocr_TSB(result)
    text, bbox = ocrINTERcorrect(row['text_origin'], texts, boxes)

    # 이 부분 추후 잘 적용되는지 확인 필요
    ocr_df.iloc[idx, -4:] = [texts_ocr, boxes_ocr, text, bbox]

In [ ]:
ocr_df.iloc[0, -4:] = [texts_ocr, boxes_ocr, text, bbox]

In [ ]:
ocr_df.iloc[0, -2:] = [texts_ocr, boxes_ocr, text, bbox]

In [ ]:
ocr_df.loc[0, 'text_ocr':] = [texts_ocr, boxes_ocr, text, bbox]

In [ ]:
png2ocr_df(ocr_df, 0, 5)

In [ ]:
ocr_df

In [ ]:
ocr_df.loc[len(ocr_df)] = [ocr_id, ocr_image, ocr_age, ocr_sex, ocr_background, ocr_text_origin, ocr_bbox_origin,
                                   None, None, None, None]

# 사양된 코드

In [ ]:
# # 자간 데이터 코드 :

# def is_handwritten(text, max_char_spacing = 5):
#     char_positions = [ord(c) for c in text]
#     spacings = [char_positions[i+1] - char_positions[i] for i in range(len(char_positions) - 1)]
#     average_spacing = sum(spacings) / len(spacings) if spacings else 0
#     return average_spacing > max_char_spacing

# # # OCR 결과에서 손글씨만 필터링

# handwritten_texts = [text for text in ocr_results if is_handwritten(text)]

In [ ]:
# # predict > texts
# # json > texts_list

# print('< 예측 | 실제 > ')
# for predict, real in zip(texts, texts_list):
#     print('{} | {}'.format(predict, real))

In [ ]:
# # ocr 모델 적용부분
# def png2ocr(img_path):
#     ocr_model = PaddleOCR(lang = 'korean')
#     result = ocr_model.ocr(img_path, cls = False)
    
#     return result

# # ocr 모델 결과를 리스트 변수로 각각 변환
# def ocr_TSB(ocr_result):
#     res = ocr_result[0]
    
#     texts = [res[i][1][0] for i in range(len(result[0]))]
#     scores = [float(res[i][1][1]) for i in range(len(result[0]))]
#     boxes = [res[i][0] for i in range(len(result[0]))]
    
#     return texts, scores, boxes
    
# # 리스트 변수에서 확률에 따라 ocr_text_num 변수로 할당
# # 기준 확률 값 혹은 갯수는 추후 변화 가능성 있음
# def ocr_TSB_prob(texts, scores, boxes):

#     ocr_text_90 = []
#     ocr_bbox_90 = []
#     ocr_text_80 = []
#     ocr_bbox_80 = []

#     for t, s, b in zip(texts, scores, boxes):
#         print(t, s, b)
#         if s >= 0.8:
#             ocr_text_80.append(t)
#             ocr_bbox_80.append(b)

#             if s >= 0.9:
#                 ocr_text_90.append(t)
#                 ocr_bbox_90.append(b)
                
#     # 진행 상황 확인용 프린트문
# #     print()
# #     print(ocr_text_90, ocr_bbox_90)
# #     print()
# #     print(ocr_text_80, ocr_bbox_80)
    
#     return ocr_text_90, ocr_bbox_90, ocr_text_80, ocr_bbox_80

# # paddle_ocr 모델 적용 후 원하는 값들을 정리하여 각 df에 추가하는 코드
# def fillup_df(idx, img_path):
#     result = png2ocr(img_path)
#     texts, scores, boxes = ocr_TSB(result)
#     ocr_text_ocr, ocr_bbox_ocr, ocr_text, ocr_bbox = ocr_TSB_prob(texts, scores, boxes)
    
#     remove_print()
    
#     # 이 부분 추후 잘 적용되는지 확인 필요
#     ocr_df.iloc[idx, -4:] = [ocr_text_90, ocr_bbox_90, ocr_text_80, ocr_bbox_80]

# # 전체 과정을 적용하는 코드
# # 이때, 에러 방지를 위해 df에 적용할 범위를 선택하여 진행함
# def png2ocr_df(ocr_df, n = 0, m = 100):
#     for idx, row in ocr_df.iterrows[n:m]():
    
#         # 진행 상황 확인용 프린트문
#         if idx % 100 == 0:
#             print(idx)
#         # 진행 상황 확인용 프린트문
# #         print(row['image'])
#         fillup_df(row['image'])

In [ ]:
#for res in result:
print(boxes[0])
print(texts[0])
print(scores[0])#, Text: {res[1]}

# Specifying font path for draw_ocr method
font_path = 'C:/Users/UserK/ds_study/DL_proj/PaddleOCR-main/doc/fonts/korean.TTF'

# imports image
img = cv2.imread(img_path) 

# reorders the color channelsa
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 

# Visualize our image and detections
# resizing display area
plt.figure(figsize = (65, 65))

# draw annotations on image
annotated = draw_ocr(img, boxes, texts, scores, font_path = font_path)

# show the image using matplotlib
plt.imshow(annotated)